### Value at Risk

For a given confidence level and a time period, Value at Risk (VaR) is the largest possible loss based on the market condition then.  \
The VaR models assume that the expected fluactuation in returns are captured by the volatilies and correlations.  

<img src="HistoricalPriceIndexvsVaR.png" alt="Price Index vs VaR" width="300"/>
Source from [Optimal MRM](https://www.youtube.com/watch?v=L5nYuDsdmU0)  

In the graph above, VaR is computed based on 2 years historical price indices. As a result, VaR only just started increasing while volatility peaked during the Global Financial crisis in 2007. VaR peaked when the index stablized at the bottom. VaR remains high until it falls out of the 2 years historical view of the crisis.   
Stress VaR is hence introduced to assess the impact of extreme price changes on a bank's portfolio:
- Measure to be calculated weekly based on a 10 days holding period at 99% confidence level
- Model risk factor input to be derived from historical data over continuous 12 months period of significant financial stress as relevant to the bank's portfolio
- Risk factor under stress market condition to be recalibrated annually

Types of stress testing model include:  
- Variance covariance
- Historical simulation
- Monte Carlo simulation
  
In this notebook, we will run historical simulation with stock price returns from Yahoo Finance between June 2007 and June 2020.  
Within these years, there were many major economic crisis such as 2007-2009 financial crisis, 2009-2019 Greek Government-debt crisis.  
Using atoti, we will create sliding windows of:
- 10 days for the stress VaR
- 260 days for Market Data Window  


In [1]:
import atoti as tt
import pandas as pd

Welcome to atoti 0.4.0!

By using this community edition, you agree with the license available at https://www.atoti.io/eula.
Browse the official documentation at https://docs.atoti.io.
Join the community at https://www.atoti.io/register.

You can hide this message by setting the ATOTI_HIDE_EULA_MESSAGE environment variable to True.


Each PnL value in the returned vector corresponds to a scenario at the same index position.

Let's start by loading the historical data into a store.  
By setting array_sep, atoti will recognize the Values column as a vector.

In [2]:
session = tt.create_session()

types = {"AmountUSD": tt.types.DOUBLE}
calibration_store = session.read_csv(
    "s3://data.atoti.io/notebooks/model-calibration/book_values_snapshot.csv",
    keys=["Book", "Stock"],
    store_name="CalibrationStore",
    array_sep=";",
    types=types,
)

calibration_store.head(10)

AmountUSD  \
Book              Stock              
US Indices        ^DJI      1000.0   
                  ^GSPC     1000.0   
US Technology CFD AAPL      -500.0   
                  GOOG       600.0   
MSCI stocks       LUKOY     -100.0   
                  OGZPY     -200.0   
                  ROSYY      200.0   

                                                                  Values  
Book              Stock                                                   
US Indices        ^DJI    doubleVector[3273]{-0.005912434756927598, ...}  
                  ^GSPC   doubleVector[3273]{-0.005347069382311664, ...}  
US Technology CFD AAPL     doubleVector[3273]{0.011044374078303498, ...}  
                  GOOG     doubleVector[3273]{0.023211776660613914, ...}  
MSCI stocks       LUKOY  doubleVector[3273]{-0.0070968325852278635, ...}  
                  OGZPY     doubleVector[3273]{0.00780197057413945, ...}  
                  ROSYY    doubleVector[3273]{0.010795350954879934, ...}

Let's also load up the scenarios. Each of the values in the above vector is the PnL for a scenario at the corresponding index.

In [3]:
scenarios = pd.read_csv(
    "https://data.atoti.io/notebooks/model-calibration/scenario_snapshot.csv"
)["Date"].to_list()
scenarios[:10]

['2007-06-05',
 '2007-06-06',
 '2007-06-07',
 '2007-06-08',
 '2007-06-11',
 '2007-06-12',
 '2007-06-13',
 '2007-06-14',
 '2007-06-15',
 '2007-06-18']

In [4]:
### Getting data from simulator
# in the event you would like to get different data set from Yahoo Finance, uncomment the below and change the year from which you would like to start from
# Refer to https://pypi.org/project/yfinance/
#
# from utils import simulator
#
# historical_data = simulator.scenarios(2007)
# book_values = historical_data[0]
# scenarios = historical_data[1]

In [5]:
cube = session.create_cube(calibration_store, "Calibration_Cube", mode="auto")

In [6]:
h = cube.hierarchies
l = cube.levels
m = cube.measures
cube

We multiply the position _AmountUSD_ with the _Values.SUM_ vector to get the hypothetical PnL values for the past years.

In [7]:
hypothetical_pnl_vector = m["AmountUSD.SUM"] * m["Values.SUM"]
m["Hypothetical PnL Vector"] = tt.agg.sum(
    hypothetical_pnl_vector, scope=tt.scope.origin(l["Stock"])
)

We can now get the hypothetical PnL vector at stock and book level.

In [9]:
cube.visualize()

Install and enable the atoti JupyterLab extension to see this widget.

We will create an arbitrary single-level static hierarchy with the dates of the historical data which we labelled as scenarios.

In [9]:
cube.create_parameter_hierarchy(
    "Scenarios", scenarios, index_measure="Date Index",
)

In [10]:
h

Hierarchies()

Since we have a corresponding hypothetical PnL for each date scenario, we can get the PnL at date by using the date index against the vector.

In [11]:
m["PnL at date"] = m["Hypothetical PnL Vector"][m["Date Index"]]

Let's do some simple verification. In the following Pivot table, we can see the PnL value for each date scenario.  
For each date, we can drill down to the books and Ticker. 

In [13]:
cube.visualize()

Install and enable the atoti JupyterLab extension to see this widget.

Let's create the stressed VaR using 10 days of historical PnL. We do so by creating a sliding window of 10 - creating a new vector that contains the 10 PnL values before the current PnL value from the vector.

In [14]:
m["Stress Sliding Vector"] = m["Hypothetical PnL Vector"][
    m["Date Index"] - 10 : m["Date Index"]
]

m["sVaR"] = tt.array.quantile(m["Stress Sliding Vector"], 0.99)

Similarly for the Market Data, we are going to create a sliding window of 260 banking days.

In [15]:
m["Sliding Vector"] = m["Hypothetical PnL Vector"][
    m["Date Index"] - 260 : m["Date Index"]
]

Now for each Date scenario, we have a list of vectors that contains 260 days worth of historical PnL.  \
Let's compute the stressed VaR for this window at 99% confidence level.

In [16]:
m["Sliding sVaR"] = tt.array.quantile(m["Sliding Vector"], 0.99)

Let's see how the Sliding sVaR compares to sVaR. 

In [18]:
cube.visualize()

Install and enable the atoti JupyterLab extension to see this widget.

Looking at the MSCI book, the overall shape of both the sliding sVaR and sVaR are generally consistent. We see sharp contrast in 2011.  
Let's look at what happened in 2011.

<img src="crisis_timeline.jpg" alt="Crisis Timeline" width="600"/>  

Source from [ABC News](https://www.abc.net.au/news/2015-06-23/greek-debt-crisis-timeline/6564930?nw=0)  

From the above charts of the events flow of Greek debt crisis, we see the similar trend as what we generated.